# Salary estimator from listings

## Setup

In [108]:
%pip install xgboost scikit-learn plotly # gensim

     ---------------------------------------- 0.0/23.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/23.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/23.3 MB ? eta -:--:--
     --------------------------------------- 0.0/23.3 MB 653.6 kB/s eta 0:00:36
     --------------------------------------- 0.1/23.3 MB 871.5 kB/s eta 0:00:27
     ---------------------------------------- 0.2/23.3 MB 1.2 MB/s eta 0:00:19
      --------------------------------------- 0.4/23.3 MB 1.9 MB/s eta 0:00:13
     - -------------------------------------- 0.6/23.3 MB 2.9 MB/s eta 0:00:08
     - -------------------------------------- 1.0/23.3 MB 3.9 MB/s eta 0:00:06
     - -------------------------------------- 1.1/23.3 MB 3.4 MB/s eta 0:00:07
     --- ------------------------------------ 2.1/23.3 MB 6.1 MB/s eta 0:00:04
     ----- ---------------------------------- 3.3/23.3 MB 8.4 MB/s eta 0:00:03
     -------- ------------------------------- 4.7/23.3 MB 11.0 MB/s eta

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [714 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-312
  creating build\lib.win-amd64-cpython-312\gensim
  copying gensim\downloader.py -> build\lib.win-amd64-cpython-312\gensim
  copying gensim\interfaces.py -> build\lib.win-amd64-cpython-312\gensim
  copying gensim\matutils.py -> build\lib.win-amd64-cpython-312\gensim
  copying gensim\nosy.py -> build\lib.win-amd64-cpython-312\gensim
  copying gensim\utils.py -> build\lib.win-amd64-cpython-312\gensim
  copying gensim\__init__.py -> build\lib.win-amd64-cpython-312\gensim
  creating build\lib.win-amd64-cpython-312\gensim\corpora
  copying gensim\corpora\bleicorpus.py -> build\lib.win-amd64-cpython-312\gensim\corpora
  copying gensim\corpora\csvcorpus.py -> build\lib.win-amd64-cpython-312\gensim\corpora
  copying gensim\corpora\dictionary.py -> build\lib.win-amd64-cpython-312\gensim\corpora
  c

First we must import our packages to manage the dataset. Then we can import the data.

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

## Get the data

Now we can download the dataset.

In [2]:
df = pd.read_csv("C:/Users/jared/Documents/dev/C964-computer-science-capstone/posting-data/postings.csv")
df.head()

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,expiry,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,1.715990e+12,NaN,NaN,Requirements: \n\nWe are seeking a College or ...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,...,1.715450e+12,NaN,NaN,NaN,1.712858e+12,NaN,0,FULL_TIME,USD,BASE_SALARY
2,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",64896719.0,8.0,NaN,...,1.715870e+12,NaN,NaN,We are currently accepting resumes for FOH - A...,1.713278e+12,NaN,0,FULL_TIME,USD,BASE_SALARY
3,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,YEARLY,"New Hyde Park, NY",766262.0,16.0,NaN,...,1.715488e+12,NaN,NaN,This position requires a baseline understandin...,1.712896e+12,NaN,0,FULL_TIME,USD,BASE_SALARY
4,35982263,NaN,Service Technician,Looking for HVAC service tech with experience ...,80000.0,YEARLY,"Burlington, IA",NaN,3.0,NaN,...,1.716044e+12,NaN,NaN,NaN,1.713452e+12,NaN,0,FULL_TIME,USD,BASE_SALARY


In [3]:
headers = list(df.columns)
print(headers)

['job_id', 'company_name', 'title', 'description', 'max_salary', 'pay_period', 'location', 'company_id', 'views', 'med_salary', 'min_salary', 'formatted_work_type', 'applies', 'original_listed_time', 'remote_allowed', 'job_posting_url', 'application_url', 'application_type', 'expiry', 'closed_time', 'formatted_experience_level', 'skills_desc', 'listed_time', 'posting_domain', 'sponsored', 'work_type', 'currency', 'compensation_type']


In [26]:
for c in df.columns:
    print(c,'----------------')
    print(df[c].unique()[:5])
    print('    nan count:',df[c].isna().sum())

job_id ----------------
[  921716  1829192 10998357 23221523 35982263]
    nan count: 0
company_name ----------------
['Corcoran Sawyer Smith' nan 'The National Exemplar '
 'Abrams Fensterman, LLP' 'Downtown Raleigh Alliance']
    nan count: 510
title ----------------
['Marketing Coordinator' 'Mental Health Therapist/Counselor'
 'Assitant Restaurant Manager'
 'Senior Elder Law / Trusts and Estates Associate Attorney'
 ' Service Technician']
    nan count: 0
description ----------------
['Job descriptionA leading real estate firm in New Jersey is seeking an administrative Marketing Coordinator with some experience in graphic design. You will be working closely with our fun, kind, ambitious members of the sales team and our dynamic executive team on a daily basis. This is an opportunity to be part of a fast-growing, highly respected real estate brokerage with a reputation for exceptional marketing and extraordinary culture of cooperation and inclusion.Who you are:You must be a well-organ

In [25]:
columns_to_drop = [
    'views','applies','original_listed_time','remote_allowed','job_posting_url','application_url','application_type',
    'expiry','closed_time','listed_time','posting_domain','sponsored','compensation_type','sponsored',
    ]
if 'views' in df.columns:
    df.drop(columns_to_drop, axis=1, inplace=True)
print([c.title() for c in df.columns])
print(df.columns)

['Job_Id', 'Company_Name', 'Title', 'Description', 'Max_Salary', 'Pay_Period', 'Location', 'Company_Id', 'Med_Salary', 'Min_Salary', 'Formatted_Work_Type', 'Formatted_Experience_Level', 'Skills_Desc', 'Work_Type', 'Currency']
Index(['job_id', 'company_name', 'title', 'description', 'max_salary',
       'pay_period', 'location', 'company_id', 'med_salary', 'min_salary',
       'formatted_work_type', 'formatted_experience_level', 'skills_desc',
       'work_type', 'currency'],
      dtype='object')


In [43]:
pay_cols = ["max_salary","med_salary","min_salary"]
df.dropna(thresh=1, subset=pay_cols, inplace=True)
len(df.index)

36073

In [27]:
df['location'].unique()

array(['Princeton, NJ', 'Fort Collins, CO', 'Cincinnati, OH', ...,
       'Middlebury, IN', 'Liberty Corner, NJ', 'Carroll County, MD'],
      dtype=object)

In [8]:
df['formatted_experience_level'].unique()

array([nan, 'Mid-Senior level', 'Associate', 'Entry level', 'Director',
       'Internship', 'Executive'], dtype=object)

In [28]:
graph_cols = pay_cols + ['company_name','title','location','pay_period']
pay_df = df[graph_cols].copy()

In [45]:
state = pay_df['location'].str.split(',').str[1]
pay_df['state'] = state
#pay_df['location'] = pay_df['location'].str.split(',')[1]
#print(pay_df)
print(pay_df.dtypes)

max_salary      float64
med_salary      float64
min_salary      float64
company_name     object
title            object
location         object
pay_period       object
state            object
dtype: object


In [66]:
salary_df = pay_df[pay_df['pay_period'] != 'HOURLY']
salary_state_df = salary_df[['state']+pay_cols]
salary_state_df['all_pay'] = salary_state_df[pay_cols].values.tolist()

C:\Users\jared\AppData\Local\Temp\ipykernel_8912\3498547486.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [69]:

print(salary_state_df.head())
salary_state_df['avg_pay'] = (salary_state_df['all_pay'][~salary_state_df['all_pay'].isna()]).mean()
print(salary_state_df.head())
fig_data = salary_state_df.groupby('state')
print(fig_data.mean())

fig_data = fig_data[['state', 'all_pay']]
print(fig_data.head())
import plotly.express as px
fig = px.bar()
fig.show()

  state  max_salary  med_salary  min_salary                    all_pay
2    OH     65000.0         NaN     45000.0    [65000.0, nan, 45000.0]
3    NY    175000.0         NaN    140000.0  [175000.0, nan, 140000.0]
4    IA     80000.0         NaN     60000.0    [80000.0, nan, 60000.0]
6   NaN    300000.0         NaN     60000.0   [300000.0, nan, 60000.0]
7    CA    120000.0         NaN     90000.0   [120000.0, nan, 90000.0]


TypeError: complex() first argument must be a string or a number, not 'list'